In [1]:
from cloudsafe import *
database = 'here-api/main.db'
conn = create_connection(database)
departures = select_all_departures(conn)
stations = select_all_stations(conn)
bus_stops = select_all_bus_stops()

In [2]:
coords = stations.iloc[0].coordinates_here.split(',')
m = folium.Map([coords[0], coords[1]], zoom_start=10)
stations[['latitude', 'longitude']] = stations.apply(get_coords, axis=1, result_type="expand")

In [3]:
# convert to (n, 2) nd-array format for heatmap
stationArr = stations[['latitude', 'longitude']].to_numpy()

In [4]:
frequencies = stations.name_overpass.value_counts()

In [5]:
rows = []
for station in frequencies.keys():
    first = stations.loc[stations['name_overpass'] == station].iloc[0]
    rows.append({'station_name': station,
                 'longitude': first.longitude,
                 'latitude': first.latitude,
                 'bus_count': frequencies.get(station)})

final = pd.DataFrame(rows)
final['longitude'] = final['longitude'].astype(float)
final['latitude'] = final['latitude'].astype(float)

top_right = [final['latitude'].max(), final['longitude'].max()]
bottom_left = [final['latitude'].min(), final['longitude'].min()]
grid = get_geojson_grid(top_right, bottom_left)

In [6]:
counts = []
for box in grid:
    upper_right = box["properties"]["upper_right"]
    lower_left = box["properties"]["lower_left"]
    items = final.loc[(final['latitude'] <= upper_right[1]) &
                      (final['longitude'] <= upper_right[0]) &
                      (final['longitude'] >= lower_left[0]) &
                      (final['latitude'] >= lower_left[1])]
    counts.append(len(items))
max_count = max(counts)

In [7]:
for i, geo_json in enumerate(grid):
    color = plt.cm.Reds(counts[i] / max_count)
    color = mpl.colors.to_hex(color)
    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, 
                        color=color: {
                            'fillColor': color,
                            'color': "black",
                            'weight': 2,
                            'dashArray': '5, 5',
                            'fillOpacity': 0.55,
                        })

    m.add_child(gj)

In [8]:
m